<a href="https://colab.research.google.com/github/sashapff/bioinformatics-algorithms/blob/master/alignment/global%26local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
def_value = 28
begin_value = 4

In [6]:
scoring_matrix = [[1,-1,-1,-1,-1], 
                  [-1,1,-1,-1,-1], 
                  [-1,-1,1,-1,-1], 
                  [-1,-1,-1,1,-1], 
                  [-1,-1,-1,-1,-10**10]]

toNum = {"A":0, "C":1, "G":2, "T":3, "_":4}

In [7]:
def global_build_path(f, s):
    n = len(f) + 1
    m = len(s) + 1
    # 1 - up, 0 - diag, -1 - left, 28 - start
    a = [[(0, def_value) for i in range(m)] for j in range(n)]
    for i in range(1, n):
        a[i][0] = (-i, 1)
    for i in range(1, m):
        a[0][i] = (-i, -1)
    for i in range(1, n):
        for j in range(1, m):
            best = (-n - m, 0)
            if a[i - 1][j - 1][0] > best[0]:
                best = (a[i - 1][j - 1][0], 0)
            if a[i - 1][j][0] > best[0]:
                best = (a[i - 1][j][0], 1)
            if a[i][j - 1][0] > best[0]:
                best = (a[i][j - 1][0], -1)
            a[i][j] = (best[0] + scoring_matrix[toNum[f[i - 1]]][toNum[s[j - 1]]], best[1])
    return a

In [8]:
def global_find_answer(f, s, a):
    n = len(f) + 1
    m = len(s) + 1
    f_ans, s_ans = "", ""
    cur_i = n - 1
    cur_j = m - 1
    while a[cur_i][cur_j][1] != def_value:
        new_cur_i, new_cur_j = cur_i, cur_j
        if a[cur_i][cur_j][1] == 0:
            f_ans += f[cur_i - 1]
            s_ans += s[cur_j - 1]
            new_cur_i = cur_i - 1
            new_cur_j = cur_j - 1
        if a[cur_i][cur_j][1] == -1:
            f_ans += '_'
            s_ans += s[cur_j - 1]
            new_cur_j = cur_j - 1
        if a[cur_i][cur_j][1] == 1:
            f_ans += f[cur_i - 1]
            s_ans += '_'
            new_cur_i = cur_i - 1
        cur_i = new_cur_i
        cur_j = new_cur_j
    f_ans = f_ans[::-1]
    s_ans = s_ans[::-1]
    return (f_ans, s_ans)

In [9]:
def local_build_path(f, s):
    n = len(f) + 1
    m = len(s) + 1
    # 1 - up, 0 - diag, -1 - left, def_value - init, begin_value - start
    a = [[(0, def_value) for i in range(m)] for j in range(n)]
    for i in range(1, n):
        a[i][0] = (0, 1)
    for i in range(1, m):
        a[0][i] = (0, -1)
    for i in range(1, n):
        for j in range(1, m):
            best = (-n - m, 0)
            if a[i - 1][j - 1][0] > best[0]:
                best = (a[i - 1][j - 1][0], 0)
            if a[i - 1][j][0] > best[0]:
                best = (a[i - 1][j][0], 1)
            if a[i][j - 1][0] > best[0]:
                best = (a[i][j - 1][0], -1)
            if best[0] < 0:
                best = (0, begin_value)
            a[i][j] = (best[0] + scoring_matrix[toNum[f[i - 1]]][toNum[s[j - 1]]], best[1])
    return a

In [10]:
def local_find_last(a):
    n = len(a)
    m = len(a[0])
    max = (n - 1, m - 1)
    for i in range(n):
        for j in range(m):
          if a[i][j][0] > a[max[0]][max[1]][0]:
            max = (i, j)
    return max

In [11]:
def local_find_answer(f, s, a):
    n = len(f) + 1
    m = len(s) + 1
    f_ans, s_ans = "", ""

    cur_i, cur_j = local_find_last(a)

    for i in range(n - 2, cur_i, -1):
        f_ans += f[i]
        s_ans += '_'
    for j in range(m - 2, cur_j, -1):
        f_ans += '_'
        s_ans += s[j]

    while a[cur_i][cur_j][1] != def_value:
        new_cur_i, new_cur_j = cur_i, cur_j
        if a[cur_i][cur_j][1] == 0:
            f_ans += f[cur_i - 1]
            s_ans += s[cur_j - 1]
            new_cur_i = cur_i - 1
            new_cur_j = cur_j - 1
        if a[cur_i][cur_j][1] == -1:
            f_ans += '_'
            s_ans += s[cur_j - 1]
            new_cur_j = cur_j - 1
        if a[cur_i][cur_j][1] == 1:
            f_ans += f[cur_i - 1]
            s_ans += '_'
            new_cur_i = cur_i - 1
        if a[cur_i][cur_j][1] == begin_value:
            break

        cur_i = new_cur_i
        cur_j = new_cur_j

    for i in range(cur_i - 1, -1, -1):
        f_ans += f[i]
        s_ans += '_'
    for j in range(cur_j - 1, -1, -1):
        f_ans += '_'
        s_ans += s[j]

    
    f_ans = f_ans[::-1]
    s_ans = s_ans[::-1]
    return (f_ans, s_ans)

In [17]:
f = "ACTGAG"
s = "GCTACT"
gf, gs = global_find_answer(f, s, global_build_path(f, s))
lf, ls = local_find_answer(f, s, local_build_path(f, s))
print("global\n")
print(gf)
print(gs)
print("\n\nlocal\n")
print(lf)
print(ls)

global

ACTGA_G
GCT_ACT


local

___ACTAG
GCTACT__
